<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - Client Side Processing

## STAC Data Fusion


In this interactive notebook we will show some usage examples of the Client Side Processing functionality of the openEO Python client.

## Requirements

To use this functionality, you need `Python>=3.9`.

You can install it using:
    `pip install openeo[localprocessing]`

<div class="alert alert-block alert-warning">
This functionality is still under development and the installation procedure might change.
Please refer to official documentation page for the most up to date instructions:
    
<a href="url">https://open-eo.github.io/openeo-python-client/cookbook/localprocessing.html</a>

## Sentinel-2 and Landsat-8 data

We are going to work with the Sentinel-2 and Landsat-8 data provided from the Microsoft Planetary Computer STAC Catalog.

In [1]:
from openeo.local import LocalConnection
local_conn = LocalConnection("./")

url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a"
spatial_extent =  {"east": 11.406212,
          "north": 46.522237,
          "south": 46.461019,
          "west": 11.259613}
temporal_extent = ["2022-06-01","2022-06-10"]
bands = ["B04","B08"]
properties = {"eo:cloud_cover": dict(lt=90)}
s2_datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    temporal_extent=temporal_extent,
                    bands=bands,
                    properties=properties)
# s2_datacube.execute()

Did not load machine learning processes due to missing dependencies: Install them like this: `pip install openeo-processes-dask[implementations, ml]`
Did not load experimental processes due to missing dependencies. Install them like this: `pip install openeo-processes-dask[implementations, experimental]`
C:\Users\mclaus\Documents\GitHub\openeo-python-client\openeo\metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")
C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:369: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [2]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2"
spatial_extent =  {"east": 11.406212,
          "north": 46.522237,
          "south": 46.461019,
          "west": 11.259613}
temporal_extent = ["2022-06-01","2022-06-10"]
bands = ["red","nir08"]
properties = {"eo:cloud_cover": dict(lt=90)}
l8_datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    temporal_extent=temporal_extent,
                    bands=bands,
                    properties=properties)
# l8_datacube.execute()

C:\Users\mclaus\Documents\GitHub\openeo-python-client\openeo\metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")
C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:369: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


We notice that the Landsat-8 data has a different size (due to the different resolution) compared to the Sentinel-2 data.

We need to combine data coming from different satellite sources with different projections and resolution.

In this scenario we could use the `resample_cube_spatial` openEO process that allows to reproject and crop the source to match a target datacube in one shot.

Have a look at the official process description if you want to know more details: https://processes.openeo.org/#resample_cube_spatial

We now want to resample the L8 data to match S2, for being able to merge them later:

In [3]:
l8_resampled = l8_datacube.resample_cube_spatial(target=s2_datacube,method="bilinear")
l8_resampled

Now our Landsat-8 data has the same spatial extent and resolution.

However, the two datacubes have different number of time steps. So, we have to align that temporal dimension as well to make a meaningful merge.

We could compute a temporal aggregation using the openEO process `aggregate_temporal_period`, to compute the monthly median values.

Have a look at the official process description if you want to know more details: https://processes.openeo.org/#aggregate_temporal_period

In [4]:
s2_monthly_median = s2_datacube.aggregate_temporal_period(period="month",reducer="median")
l8_monthly_median = l8_resampled.aggregate_temporal_period(period="month",reducer="median")

In [ ]:
s2_monthly_median_xr = s2_monthly_median.execute()
l8_monthly_median_xr = l8_monthly_median.execute()

Now that spatial and temporal dimensions are aligned, we can proceed merging the two datacubes and look at the generated openEO process graph:

In [5]:
s2_l8_monthly_median = s2_monthly_median.merge_cubes(l8_monthly_median)
s2_l8_monthly_median

Finally execute the process and check the output:

In [6]:
s2_l8_monthly_median_xarray = s2_l8_monthly_median.execute()
s2_l8_monthly_median_xarray

Deserialised process graph into nested structure


Walking node root-9b5d676e-d3d1-4517-b3c3-04fbf793acbb
Walking node aggregatetemporalperiod1-9b5d676e-d3d1-4517-b3c3-04fbf793acbb
Walking node median1-fbf3b4ff-917d-4bb3-9364-7c35ee8b01e2
Walking node loadstac1-9b5d676e-d3d1-4517-b3c3-04fbf793acbb
Walking node aggregatetemporalperiod2-9b5d676e-d3d1-4517-b3c3-04fbf793acbb
Walking node median2-71239974-7161-4379-9676-ba7f7b7ec6fa
Walking node resamplecubespatial1-9b5d676e-d3d1-4517-b3c3-04fbf793acbb
Walking node loadstac2-9b5d676e-d3d1-4517-b3c3-04fbf793acbb
Walking node loadstac1-9b5d676e-d3d1-4517-b3c3-04fbf793acbb


C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:369: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
C:\Users\mclaus\Documents\GitHub\stackstac\stackstac\prepare.py:369: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


type: conflicting sizes for dimension 'band': length 4 on 'band' and length 2 on {'x': 'x', 'y': 'y', 'band': 'title', 'time': 'time'}

The output now contains the red and nir bands of Sentinel-2 and Landsat-8.

The datacube is aligned spatially and temporally, which was possible using the `resample_cube_spatial` and `aggregate_temporal_period` openEO processes.

In [ ]:
s2_l8_monthly_median_xarray.compute()